# CONNECT 4

## Informações Gerais

**Unidade Curricular:** Inteligência Artificial

**Turma:** PL6

**Grupo 2:**
- Hugo Duarte de Sousa — nº 202305900
- Isabela Britto Cartaxo — nº 202300339
- Tiago Figueredo Silva — nº 202000612

## Sumário

- [1. Introdução](#1-introducao)
- [2. Objetivos](#2-objetivos)
- [3. Lógica do Jogo e Estrutura do Tabuleiro](#3-logica-do-jogo-e-estrutura-do-tabuleiro)
- [4. Ciclo do Jogo e Modos de Interação](#4-ciclo-do-jogo-e-modos-de-interacao)


## 1. Introdução

Este trabalho foi desenvolvido no âmbito da unidade curricular de Inteligência Artificial (2024/2025) e tem como principal finalidade a aplicação de estratégias de procura adversária e algoritmos de aprendizagem supervisionada, com enfoque nas árvores de decisão. O projeto encontra-se dividido em duas componentes principais:

A implementação do algoritmo Monte Carlo Tree Search (MCTS) com a fórmula UCT, aplicado ao jogo Connect Four (Quatro em Linha), um clássico de estratégia entre dois jogadores.

A construção de um classificador baseado em árvores de decisão, recorrendo ao algoritmo ID3, treinado com dois conjuntos de dados distintos: o conhecido dataset Iris e um dataset gerado a partir de simulações com o MCTS no jogo Connect Four.

Este trabalho tem como intuito consolidar os conhecimentos teóricos abordados nas aulas, promovendo a sua aplicação prática em contextos desafiantes de tomada de decisão, planeamento e aprendizagem automática.

### O jogo Connect Four
O Connect Four (ou Quatro em Linha) é um jogo de estratégia para dois jogadores, jogado num tabuleiro vertical com 7 colunas e 6 linhas. Cada jogador tem 21 peças da sua cor (normalmente vermelho e amarelo), e o objetivo do jogo é ser o primeiro a alinhar quatro peças consecutivas na horizontal, vertical ou diagonal.

Os jogadores jogam alternadamente, inserindo uma peça numa das colunas. A peça cai até ocupar a posição mais baixa disponível nessa coluna. O jogo termina quando um dos jogadores vence ou quando o tabuleiro está cheio, resultando num empate.

## 2. Objetivos

- Implementar um agente capaz de jogar Connect Four utilizando o algoritmo MCTS com UCT.

- Suportar diferentes modos de jogo: humano vs. humano, humano vs. computador e computador vs. computador (com estratégias distintas).

- Implementar o algoritmo ID3 para gerar árvores de decisão a partir de dados, sem recorrer a bibliotecas externas como o scikit-learn.

- Avaliar a performance das árvores de decisão utilizando:

- O dataset Iris.

- Um dataset gerado artificialmente a partir da execução do MCTS no Connect Four, contendo pares (estado do jogo, jogada recomendada).

- Documentar as decisões tomadas, discutir os resultados obtidos.

## 3. Lógica do Jogo e Estrutura do Tabuleiro

A lógica do jogo Connect Four foi implementada através da classe Board, que representa o estado atual do tabuleiro, permite realizar jogadas, verificar jogadas legais e determinar se o jogo terminou com vitória ou empate.

### Estrutura do tabuleiro
O tabuleiro é representado por uma matriz de 6 linhas e 7 colunas (6x7), modelada como uma lista de listas de strings. Cada posição pode conter:

- "." – casa vazia

- "X" – peça do jogador 1

- "O" – peça do jogador 2

A matriz é inicializada com todas as casas vazias, e as peças "caem" até à posição mais baixa disponível em cada coluna, respeitando as regras do jogo real.

### Componentes principais da implementação
- self.board: Matriz 6x7 que guarda o estado atual do tabuleiro.

- self.y_coords: Dicionário que indica a próxima linha disponível em cada coluna, permitindo simular a gravidade.

- self.counter: Contador de jogadas realizadas (máximo 42).

- make_move(x, current_player): Realiza uma jogada válida, atualizando o tabuleiro e o estado interno.

- is_legal_move(x): Verifica se a jogada na coluna x é legal (coluna dentro dos limites e com espaço disponível).

- is_board_full(): Verifica se o tabuleiro está cheio.

- is_won(x, current_player): Verifica se a última jogada feita na coluna x resultou numa vitória para o jogador atual.

- is_tie(): Verifica se o jogo terminou empatado.

- get_possible_moves(current_player): Devolve todos os possíveis estados do tabuleiro resultantes de uma jogada válida do jogador atual (útil para o algoritmo MCTS).

In [1]:
from copy import deepcopy

class Board:
    def __init__(self):
        """
        Initializes the Connect Four board.
        - 6 rows x 7 columns represented by a list of lists.
        - y_coords stores the next available row index for each column.
        """
        self.counter = 0
        self.board_width = 7
        self.board_height = 6
        self.board = [list(".......") for _ in range(self.board_height)]
        self.y_coords = {col: self.board_height - 1 for col in range(self.board_width)}
        self.last_move_column = None

    def to_tuple(self):
        """
        Returns an immutable representation of the board (useful for hashing).
        """
        return tuple(tuple(row) for row in self.board)

    def get_simulation_board(self):
        """
        Placeholder for compatibility or extensions (e.g., for neural network input).
        """
        return []

    def print_board(self):
        """
        Displays the board to the console.
        """
        print("\n" + " ".join(map(str, [1, 2, 3, 4, 5, 6, 7])))
        for row in self.board:
            print(" ".join(row))
        print()

    def is_empty(self, x, y) -> bool:
        """
        Checks if a given cell is empty.
        """
        return self.board[y][x] == "."

    def is_board_full(self) -> bool:
        """
        Checks if the board is full (42 moves).
        """
        return self.counter == 42

    def is_legal_move(self, x) -> bool:
        """
        Checks if a move is legal:
        - Column must be within bounds.
        - Column must not be full.
        """
        if self.is_board_full():
            return False
        if not 0 <= x < self.board_width:
            return False
        if self.y_coords[x] < 0:
            return False
        return self.is_empty(x, self.y_coords[x])

    def make_move(self, x, current_player):
        """
        Places a piece from the current player in the specified column.
        Updates the board, move counter, and last played column.
        """
        y = self.y_coords[x]
        self.board[y][x] = current_player
        self.counter += 1
        self.y_coords[x] -= 1
        self.last_move_column = x

    def count_in_direction(self, current_player, dx, dy, x, y):
        """
        Counts how many consecutive pieces exist in a given direction from (x, y).
        """
        count = 0
        while (0 <= x + dx < self.board_width and 
               0 <= y + dy < self.board_height and 
               self.board[y + dy][x + dx] == current_player):
            x += dx
            y += dy
            count += 1
        return count

    def is_won(self, x, current_player) -> bool:
        """
        Checks whether the last move in column x resulted in a win.
        """
        directions = [((0, 1), (0, -1)),      # Horizontal
                      ((1, 0), (-1, 0)),      # Vertical
                      ((1, 1), (-1, -1)),     # Main diagonal
                      ((1, -1), (-1, 1))]     # Anti-diagonal

        y = self.y_coords[x] + 1  # Find the row where the last move was placed
        if y is None:
            return False
        for (dy1, dx1), (dy2, dx2) in directions:
            total = (self.count_in_direction(current_player, dx1, dy1, x, y) +
                     self.count_in_direction(current_player, dx2, dy2, x, y) + 1)
            if total >= 4:
                return True
        return False

    def has_winner(self) -> bool:
        """
        Scans the board to detect if there is a winning sequence for any player.
        Used in end-of-game checks (e.g., for tie).
        """
        for y in range(self.board_height):
            for x in range(self.board_width):
                if self.board[y][x] in ["X", "O"]:
                    current_player = self.board[y][x]
                    for (dx1, dy1), (dx2, dy2) in [((0, 1), (0, -1)),
                                                  ((1, 0), (-1, 0)),
                                                  ((1, 1), (-1, -1)),
                                                  ((1, -1), (-1, 1))]:
                        count = 1
                        count += self.count_in_direction(current_player, dx1, dy1, x, y)
                        count += self.count_in_direction(current_player, dx2, dy2, x, y)
                        if count >= 4:
                            return True
        return False

    def is_tie(self) -> bool:
        """
        Returns True if the board is full and there is no winner.
        """
        return self.is_board_full() and not self.has_winner()

    def get_possible_moves(self, current_player):
        """
        Returns a list of possible future board states given the current player's move.
        Useful for MCTS simulation.
        """
        possible_boards = []
        for i in range(self.board_width):
            if self.is_legal_move(i):
                new_board = deepcopy(self)   # NOT USE DEEP COPY, RETURN POSSIBLE MOVES INSTEAD, I.E. [1,3,4,5,6]
                new_board.make_move(i, current_player)
                possible_boards.append(new_board)
        return possible_boards


## 4. Ciclo do Jogo e Modos de Interação

Nesta secção, implementamos o ciclo principal de jogo do Connect Four, incluindo os três modos de interação disponíveis:

- **Humano vs Humano**: dois jogadores humanos alternam jogadas, inserindo a coluna desejada. O programa valida se a jogada é legal e permite repetir em caso de erro. Também é possível pedir uma sugestão de jogada, gerada pela IA com base no algoritmo MCTS.

- **IA vs Humano**: o jogador humano enfrenta um agente inteligente baseado no algoritmo **Monte Carlo Tree Search (MCTS)**. A IA calcula a melhor jogada possível em cada turno. A jogabilidade é alternada, e o tempo de execução da decisão da IA é exibido ao utilizador.

- **IA vs IA**: duas instâncias da IA jogam entre si, cada uma utilizando MCTS para escolher as suas jogadas. O jogo corre de forma automática e os tempos de decisão são registados e apresentados.

Além dos modos de jogo, também estão disponíveis funções para **geração de datasets**, que simulam partidas completas para fins de treino de modelos de aprendizagem supervisionada, nomeadamente árvores de decisão com o algoritmo ID3:

- **ai_vs_ai_simulation_generator()**: simula jogos entre IAs, gravando o estado do tabuleiro, o jogador da vez e a jogada ótima sugerida.

### Lógica do ciclo de jogo

O ciclo de jogo é executado num laço while que termina quando:
- o tabuleiro está cheio (is_board_full()), ou
- um jogador vence (is_won()), ou
- um empate é detetado (is_tie()).

Em cada iteração:
1. O estado atual do tabuleiro é impresso (print_board()).
2. O jogador da vez é definido.
3. Dependendo do modo, a jogada é obtida via input humano ou decisão da IA.
4. A jogada é validada e executada.
5. O jogo verifica se terminou.

### Tratamento de erros e robustez

Para garantir uma experiência de jogo mais segura e intuitiva, foram implementadas verificações rigorosas de input nos momentos críticos da interação com o utilizador:

- Na escolha do modo de jogo, o sistema rejeita qualquer valor inválido (como letras, símbolos ou números fora do intervalo 1–3), solicitando ao utilizador que introduza uma opção válida.

- Durante as jogadas humanas, o programa valida se a entrada corresponde a um número entre 1 e 7. Caso contrário, uma mensagem de erro é exibida e o jogador é convidado a tentar novamente, sem perder a vez.

- O programa também impede jogadas em colunas já cheias, apresentando um aviso e aguardando uma nova escolha válida.

### Execução

Este é o menu principal do programa. Existem três modos de jogo distintos que podem ser seleccionados ao executar a função run().  
No entanto, como o Jupyter Notebook **não permite input interativo direto**, para demonstrar o funcionamento do algoritmo MCTS será utilizada a função ai_vs_ai(x_simulation_limit, o_simulation_limit), que executa uma partida automática entre duas IAs.


In [2]:
import time
import numpy as np

def ai_vs_ai_simulation_generator(x_simulation_limit=10000, o_simulation_limit=10000):
    """
    Simulates a Connect Four game between two AI agents using MCTS with configurable simulation limits.

    Returns:
    - boardStates: A list of flattened 1D arrays representing the state of the board at each move
    - playerTurns: A list of strings ("X" or "O") indicating the player who made each move
    - optimalMoves: A list of integers indicating the column (1-based) chosen at each move
    """
    game = Board()
    players = ["X", "O"]
    current_index = 0
    game_over = False

    boardStates = []
    playerTurns = []
    optimalMoves = []

    while not game.is_board_full() and not game_over:
        current_player = players[current_index]

        sim_limit = x_simulation_limit if current_player == "X" else o_simulation_limit
        root = Node(game, None)
        mcts = MCTS(root, current_player, sim_limit)
        start = time.time()
        best_node = mcts.best_move()
        move = best_node.board.last_move_column
        end = time.time()

        # Register move and state
        flat_board = np.array(game.board).flatten().tolist()
        boardStates.append(flat_board)
        playerTurns.append(current_player)
        optimalMoves.append(move + 1)  # 1-based index for readability

        game.make_move(move, current_player)

        game_over = game.is_won(move, current_player)
        current_index = 1 - current_index

    return boardStates, playerTurns, optimalMoves

def human_play(game, current_player):
    """
    Handles human player's input. Allows hint request or direct column input.
    Ensures only valid numeric input in the range 1–7.
    """
    while True:
        print("Make a move by choosing your coordinates to play.")
        print("Enter column (1-7) or type 'hint' for a suggestion:")
        user_input = input()

        if user_input.lower() == 'hint':
            best_move = get_hint(game, current_player)
            print("Best move according to the AI: ", best_move)
            continue  # Ask again after showing the hint

        if not user_input.isdigit():
            print("Invalid input. Please enter a number between 1 and 7.")
            continue

        move = int(user_input) - 1
        if 0 <= move < 7:
            return move
        else:
            print("Column out of range. Try again.")


def get_hint(game, current_player):
    """
    Uses MCTS to provide a hint for the current player.
    """
    root = Node(game, None)
    mcts = MCTS(root, current_player)
    best_node = mcts.best_move()
    return best_node.board.last_move_column + 1


def human_vs_human():
    """
    Human vs. Human game loop.
    """
    game = Board()
    player = "X"
    game_over = False

    while not game.is_board_full() and not game_over:
        player = "O" if player == "X" else "X"
        game.print_board()
        print(f"It is now {player}'s turn!")

        valid_move = False
        while not valid_move:
            col = human_play(game, player)
            if not game.is_legal_move(col):
                print("\nWarning: Invalid move. Try again!")
            else:
                game.make_move(col, player)
                valid_move = True

        game_over = game.is_won(col, player)

    game.print_board()
    print("It's a tie!" if game.is_tie() else f"Player {player} has won!\n")


def ai_vs_human():
    """
    AI vs. Human game loop.
    """
    game = Board()
    ai = "O"
    human = "X"
    human_turn = False
    game_over = False

    while not game.is_board_full() and not game_over:
        current_player = human if human_turn else ai
        game.print_board()
        print(f"It is now {current_player}'s turn!")

        if human_turn:
            valid_move = False
            while not valid_move:
                move = human_play(game, current_player)
                if not game.is_legal_move(move):
                    print("\nWarning: Invalid move. Try again!")
                else:
                    game.make_move(move, current_player)
                    valid_move = True
        else:
            root = Node(game, None)
            mcts = MCTS(root, ai)
            start = time.time()
            best_node = mcts.best_move()
            move = best_node.board.last_move_column
            game.make_move(move, ai)
            end = time.time()
            print(f"AI chose column: {move + 1}\nTime taken: {end - start:.2f}s")
        game_over = game.is_won(move, current_player)
        human_turn = not human_turn

    game.print_board()
    print("It's a tie!" if game.is_tie() else f"Player {current_player} has won!\n")


def ai_vs_ai(x_simulation_limit=10000, o_simulation_limit=10000):
    """
    AI vs. AI game loop.
    """
    game = Board()
    players = ["X", "O"]
    current_index = 0
    game_over = False

    while not game.is_board_full() and not game_over:
        current_player = players[current_index]
        game.print_board()
        print(f"It is now {current_player}'s turn!")

        sim_limit = x_simulation_limit if current_player == "X" else o_simulation_limit
        root = Node(game, None)
        mcts = MCTS(root, current_player, sim_limit)
        start = time.time()
        best_node = mcts.best_move()
        move = best_node.board.last_move_column
        game.make_move(move, current_player)
        print(move)
        end = time.time()

        print(f"{current_player} chose column: {move + 1}\nTime taken: {end - start:.2f}s")
        game_over = game.is_won(move, current_player)
        current_index = 1 - current_index

    game.print_board()
    print("It's a tie!" if game.is_tie() else f"Player {current_player} has won!\n")

def run():
    """
    Entry point to choose game mode. Validates input.
    """
    print("Choose a game mode:")
    print("1. Human vs Human")
    print("2. AI vs Human")
    print("3. AI vs AI")

    while True:
        user_input = input("Enter the game mode number (1-3): ")
        if user_input.isdigit():
            game_mode = int(user_input)
            if game_mode in [1, 2, 3]:
                break
        print("Invalid game mode. Please enter 1, 2 or 3.")

    if game_mode == 1:
        human_vs_human()
    elif game_mode == 2:
        ai_vs_human()
    elif game_mode == 3:
        ai_vs_ai()



## 5. Monte Carlo Tree Search

Nesta secção, apresentamos a implementação do algoritmo **Monte Carlo Tree Search (MCTS)**, utilizado como estratégia de decisão para o jogo Connect Four.

O MCTS baseia-se em simulações aleatórias para explorar o espaço de estados do jogo e selecionar as jogadas mais promissoras. A cada iteração, o algoritmo executa quatro fases principais:

1. **Seleção**: navega pela árvore de decisão até um nó que ainda não esteja totalmente expandido.
2. **Expansão**: adiciona um novo nó filho a partir de uma jogada possível ainda não explorada.
3. **Simulação**: executa uma partida aleatória a partir do novo estado até ao final do jogo.
4. **Retropropagação**: propaga o resultado da simulação para os nós ancestrais, atualizando os valores de vitória e visita.


### Estrutura do Nó (Node)

A classe Node representa um estado na árvore de MCTS. Cada nó contém o estado atual do tabuleiro, uma ligação ao nó pai, os seus filhos, e estatísticas que guiam o algoritmo de seleção.


In [3]:
import math

class Node:
    def __init__(self, board, parent=None):
        """
        Initializes a node representing a game state in the MCTS tree.

        Parameters:
        - board: The game board associated with this node.
        - parent: The parent node in the tree (None if this is the root).
        """
        self.board = board
        self.parent = parent
        self.children = []
        self.wins = 0
        self.visits = 0

    def is_fully_expanded(self) -> bool:
        """
        Returns True if all possible moves from this state have been expanded into children.
        """
        return len(self.children) == len(self.board.get_possible_moves("X"))  # or "O" if alternating

    def best_child(self):
        """
        Returns the child node with the highest UCT (Upper Confidence Bound for Trees) value.
        Used to select the most promising node during the selection phase of MCTS.
        """
        return max(self.children, key=lambda child: child.get_uct_value())

    def add_child(self, child):
        """
        Adds a new child node to this node.
        """
        self.children.append(child)

    def get_uct_value(self, c=1.4) -> float:
        """
        Calculates the UCT value for this node, balancing exploration and exploitation.
        """
        exploitation = self.wins / (self.visits + 1e-6)
        exploration = c * math.sqrt(math.log(self.parent.visits + 1) / (self.visits + 1e-6))
        return exploitation + exploration


### Estrutura do MCTS

A classe MCTS encapsula toda a lógica do algoritmo. Abaixo, destacamos os principais métodos implementados:

- selection()  
  Seleciona o melhor caminho atual com base nos valores de UCT.

- expansion(node)  
  Cria todos os filhos possíveis para um dado nó, representando as próximas jogadas válidas.

- simulation(node)  
  Executa uma simulação aleatória até o fim do jogo, retornando o vencedor ou empate.

- backpropagation(node, result)  
  Atualiza estatísticas (vitórias e visitas) com base no resultado da simulação.

- check_for_win(leaf)  
  Verifica se existe uma jogada imediata que leva à vitória — se existir, executa essa jogada diretamente (acelera o processo).

- best_move()  
  Executa o ciclo completo do MCTS (seleção, expansão, simulação, retropropagação) um número de vezes definido por simulation_limit, e retorna o melhor nó filho da raiz.


In [4]:
import random
from copy import deepcopy

class MCTS:
    def __init__(self, initial_state, current_player, simulation_limit=10000):
        """
        Initializes the MCTS agent.

        Parameters:
        - initial_state: The root node containing the current game state.
        - current_player: The player for whom the move is being calculated.
        - simulation_limit: Maximum number of simulations to run (default: 10000).
        """
        self.root = initial_state
        self.simulation_limit = simulation_limit
        self.current_player = current_player

    def selection(self):
        """
        Selects the most promising node by traversing the tree
        using the UCT value until a leaf is reached.
        """
        node = self.root
        while node.children:
            node = node.best_child()
        return node

    def expansion(self, node):
        """
        Expands the given node by generating all possible child states.
        """
        current_player = self.current_player
        for board in node.board.get_possible_moves(current_player):
            new_node = Node(board, parent=node)
            node.add_child(new_node)

    def simulation(self, node):
        """
        Simulates a random playout from the current node until
        the game ends with a win or a tie.
        """
        sim_board = deepcopy(node.board)
        player = "O" if self.current_player == "X" else "X"

        while not sim_board.is_board_full():
            legal_moves = [i for i in range(sim_board.board_width) if sim_board.is_legal_move(i)]
            move = random.choice(legal_moves)
            sim_board.make_move(move, player)

            if sim_board.is_won(move, player):
                return player

            player = "O" if player == "X" else "X"

        return "."

    def backpropagation(self, node, result):
        """
        Updates the statistics of the nodes on the path from the
        simulation result back to the root.
        """
        while node is not None:
            node.visits += 1
            if result == self.current_player:
                node.wins += 1
            elif result == ".":
                node.wins += 0.5
            node = node.parent

    def check_for_win(self, leaf) -> Node:
        """
        Checks if there is an immediate winning move for the current player.
        If found, returns a new node representing that winning move.
        """
        possible_moves = [i for i in range(leaf.board.board_width) if leaf.board.is_legal_move(i)]

        for i in possible_moves:
            simulated_board = deepcopy(leaf.board)
            simulated_board.make_move(i, self.current_player)
            if simulated_board.is_won(i, self.current_player):
                return Node(simulated_board, parent=leaf)

        return None

    def best_move(self):
        """
        Runs the full MCTS process and returns the best child of the root node.
        """
        leaf = self.selection()
        if not leaf.children:
            self.expansion(leaf)

        winning_node = self.check_for_win(leaf)
        if winning_node:
            return winning_node

        for child in leaf.children:
            result = self.simulation(child)
            self.backpropagation(child, result)

        children = leaf.children

        for _ in range(7, self.simulation_limit + 1):
            selected_leaf = random.choice(children)
            result = self.simulation(selected_leaf)
            self.backpropagation(selected_leaf, result)

        return self.root.best_child()


### Execução da simulação

Este é um exemplo de demonstração de uma simulação entre dois agentes inteligentes baseados no algoritmo MCTS, utilizando a função ai_vs_ai(). Nesta função, o número de simulações por jogada pode ser ajustado através dos parâmetros x_simulation_limit e o_simulation_limit, permitindo observar o impacto da profundidade da busca no desempenho de cada IA.

Sinta-se à vontade para experimentar diferentes valores de limite de simulação para comparar os tempos de execução e a qualidade das decisões tomadas por cada agente.

In [5]:
ai_vs_ai(x_simulation_limit=10000, o_simulation_limit=10000)


1 2 3 4 5 6 7
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

It is now X's turn!
3
X chose column: 4
Time taken: 1.90s

1 2 3 4 5 6 7
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .

It is now O's turn!
3
O chose column: 4
Time taken: 1.77s

1 2 3 4 5 6 7
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
. . . X . . .

It is now X's turn!
2
X chose column: 3
Time taken: 1.62s

1 2 3 4 5 6 7
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
. . X X . . .

It is now O's turn!
4
O chose column: 5
Time taken: 1.45s

1 2 3 4 5 6 7
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
. . X X O . .

It is now X's turn!
0
X chose column: 1
Time taken: 1.52s

1 2 3 4 5 6 7
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
X . X X O . .

It is now O's turn!
1
O chose column: 2
Time taken: 1.22s

1 2 3 4 5 6 7
. . . . . . .
. . . . . . .
. . . . .

## 6. Geração do Dataset — Connect Four IA vs IA (MCTS)

Este script simula partidas completas de Connect Four entre dois agentes de IA utilizando o algoritmo MCTS (Monte Carlo Tree Search). Cada jogada é escolhida com base em 5.000 simulações (valor configurável).

Ao todo, são gerados 500 jogos, e os dados resultantes são gravados em um ficheiro .csv estruturado, ideal para treino ou análise posterior.

### Estrutura do ficheiro CSV
O ficheiro gerado contém 44 colunas no total:

- Colunas 1 a 42: Estado do tabuleiro achatado (6x7) — preenchido linha a linha

- Coluna 43: player_turn — o jogador que fez a jogada ("X" ou "O")

- Coluna 44: chosen_move — coluna escolhida pelo algoritmo MCTS (base 1, de 1 a 7)

Cada linha representa uma jogada dentro de um jogo. 

In [8]:
import numpy as np
import csv
import os

def run_simulation(simulation_limit):
    """
    Executes a single simulation of a Connect Four match using two AI agents (MCTS).

    Returns:
    - boardStates: A list of 1D arrays representing the state of the board at each move
    - playerTurns: A list indicating which player ("X" or "O") made each move
    - optimalMoves: A list of the moves chosen by the agents at each step
    """
    boardStates, playerTurns, optimalMoves = ai_vs_ai_simulation_generator(simulation_limit, simulation_limit)
    return boardStates, playerTurns, optimalMoves

def generate_db_csv(folder="datasets", filename="connect4_dataset.csv", iterations=150, append=True, simulation_limit=10000):
    """
    Generates a dataset by simulating multiple Connect Four games and logging the results
    into a CSV file inside a specific folder.

    Parameters:
    - folder: Folder to store the CSV file (e.g., "datasets" or "data")
    - filename: Name of the CSV file
    - iterations: Number of games to simulate
    - append: Whether to append to the existing file or overwrite it
    """
    os.makedirs(folder, exist_ok=True)  # Create folder if it doesn't exist
    filepath = os.path.join(folder, filename)
    
    file_exists = os.path.isfile(filepath)
    mode = 'a' if append else 'w'

    with open(filepath, mode, newline='') as csvfile:
        writer = csv.writer(csvfile)

        # Write header if file is new or being overwritten
        if not file_exists or not append:
            header = [f"cell_{i+1}" for i in range(42)] + ["player_turn", "chosen_move"]
            writer.writerow(header)

        for i in range(iterations):
            print(f"Initiating simulation {i + 1}:")
            boardStates, playerTurns, optimalMoves = run_simulation(simulation_limit)

            for state, player, move in zip(boardStates, playerTurns, optimalMoves):
                row = list(state) + [player, move]
                writer.writerow(row)

            print(f"Simulation {i + 1} completed!")
            print(f"Progress: { round(((i + 1)/iterations) * 100, 2)}%")
            print("")

### Exemplo de geração do dataset

Abaixo, apresentamos um exemplo de como o dataset foi gerado utilizando partidas entre dois agentes de IA com o algoritmo MCTS (Monte Carlo Tree Search).

Para fins de demonstração, limitamos a geração a apenas 5 jogos, pois simular as 500 partidas completas leva um tempo considerável devido à complexidade do algoritmo (5.000 simulações por jogada).

No entanto, se desejar gerar um dataset maior, basta alterar o valor do parâmetro iterations na função generate_db_csv.

**Nota**: O dataset utilizado para treinar o Decision Tree foi gerado com 500 simulações, garantindo uma base mais robusta e representativa para o aprendizado do modelo.

In [9]:
generate_db_csv(folder="datasets", filename="connect4_dataset_example.csv", iterations=5, append=True, simulation_limit=5000)

Initiating simulation 1:
Simulation 1 completed!
Progress: 20.0%

Initiating simulation 2:
Simulation 2 completed!
Progress: 40.0%

Initiating simulation 3:
Simulation 3 completed!
Progress: 60.0%

Initiating simulation 4:
Simulation 4 completed!
Progress: 80.0%

Initiating simulation 5:
Simulation 5 completed!
Progress: 100.0%



# 7. Decision Tree

Após gerar nosso dataset com simulações de partidas de Connect Four utilizando MCTS, o próximo passo é construir um modelo que consiga aprender os padrões dessas jogadas e tomar decisões semelhantes. Para isso, vamos utilizar uma árvore de decisão, um dos algoritmos mais interpretáveis e intuitivos da aprendizagem supervisionada.

A árvore de decisão será treinada para prever a melhor jogada (coluna) com base no estado atual do tabuleiro e no jogador da vez. Cada nó da árvore fará uma divisão (split) nos dados com base em um dos atributos do tabuleiro, buscando separar os exemplos da forma mais "pura" possível (por exemplo, agrupando situações que levam a uma mesma jogada ideal).

### Estrutura do Nó (Node)

Antes de construir a árvore propriamente dita, é importante definir como será representado cada nó da árvore. Para isso, criamos a classe Node_DT.

Essa classe representa um único ponto de decisão ou previsão na árvore:

In [1]:
class NodeDT:
    def __init__(self, feature=None, split_criteria=None, left=None, right=None, value=None):
        """
        Initializes a node for a Decision Tree.

        Parameters:
        - feature: The feature used to split the data at this node (e.g., "number of rooms").
        - split_criteria: The threshold value for splitting (e.g., <= 5 goes left, > 5 goes right).
        - left: The left child node (corresponds to data satisfying the split condition).
        - right: The right child node (corresponds to data not satisfying the split condition).
        - value: The predicted value or class if this node is a leaf.
        """
        self.feature = feature
        self.split_criteria = split_criteria
        self.left = left
        self.right = right
        self.value = value
        self.children = {}  # For potential extension to multi-branch trees (e.g., categorical splits)

    def is_leaf(self) -> bool:
        """
        Returns True if the node is a leaf (i.e., it holds a prediction value and has no children).
        """
        return self.value is not None


### Estrutura do classificador decision tree

Nesta etapa, implementamos a classe DecisionTree, responsável por aprender as melhores jogadas a partir do dataset gerado com partidas entre agentes MCTS.

A árvore é construída de forma recursiva, dividindo os dados com base no ganho de informação até atingir a profundidade máxima ou outros critérios de parada.

Principais métodos:

- grow_tree(): Função principal que constrói a árvore recursivamente.

- best_split(): Itera sobre os atributos disponíveis e retorna a melhor divisão possível.

- information_gain(): Calcula o ganho de informação para a divisão escolhida.

- split(): Função auxiliar que separa os dados em subconjuntos com base no atributo escolhido.

- entropy(): Função auxiliar que retorna a entropia do conjunto de dados.

- predict(): Função auxiliar que percorre a árvore para fazer previsões.

- traverse_tree(): Função recursiva que percorre a árvore para um exemplo de teste e retorna a predição.

In [2]:
from collections import Counter

class DecisionTree:
    def __init__(self, min_sample_split=2, max_depth=15, n_features=None) -> None:
        """
        Initializes the Decision Tree classifier.

        Parameters:
        - min_sample_split: Minimum number of samples required to split an internal node.
        - max_depth: Maximum depth allowed for the tree.
        - n_features: Number of features to consider when looking for the best split.
        """
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        """
        Fits the Decision Tree to the training data by building the tree recursively.
        """
        self.n_features = X.shape[1]
        self.root = self.grow_tree(X, y)

    def grow_tree(self, X, y, depth=0):
        """
        Recursively builds the decision tree.
        """
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # Stopping conditions
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_sample_split):
            leaf_value = self.most_common_label(y)
            return NodeDT(value=leaf_value)

        best_feature, best_value = self.best_split(X, y, list(range(n_feats)))

        node = NodeDT(feature=best_feature)

        for value in np.unique(X.iloc[:, best_feature]):
            idxs = self.split(X.iloc[:, best_feature], value)
            if len(idxs) > 0:
                node.children[value] = self.grow_tree(
                    X.iloc[idxs, :],
                    y.iloc[idxs],
                    depth + 1
                )

        return node

    def most_common_label(self, y):
        """
        Returns the most common label in the target array y.
        """
        counter = Counter(y)
        value = counter.most_common(1)
        if value:
            return value[0][0]
        return value

    def best_split(self, X, y, feat_idxs):
        """
        Finds the feature and value that provide the best information gain.
        """
        if feat_idxs is None:
            feat_idxs = range(X.shape[1])

        best_gain = -1
        best_split_feat, best_split_value = None, None

        for feat_idx in feat_idxs:
            X_column = X.iloc[:, feat_idx]
            limits = np.unique(X_column)

            for split_criteria in limits:
                gain = self.information_gain(y, X_column, split_criteria)

                if gain > best_gain:
                    best_gain = gain
                    best_split_feat = feat_idx
                    best_split_value = split_criteria

        return best_split_feat, best_split_value

    def information_gain(self, y, X_column, split_criteria):
        """
        Computes the information gain of a split.
        """
        parent_entropy = self.entropy(y)
        matching_idxs = self.split(X_column, split_criteria)
        non_matching_idxs = np.setdiff1d(np.arange(len(X_column)), matching_idxs)

        if len(matching_idxs) == 0 or len(non_matching_idxs) == 0:
            return 0

        n = len(y)
        n_matching = len(matching_idxs)
        n_non_matching = len(non_matching_idxs)

        e_matching = self.entropy(y.iloc[matching_idxs])
        e_non_matching = self.entropy(y.iloc[non_matching_idxs])

        child_entropy = (n_matching / n) * e_matching + (n_non_matching / n) * e_non_matching

        information_gain = parent_entropy - child_entropy

        return max(0, information_gain)

    def split(self, X_column, value):
        """
        Returns the indices of samples where the feature matches the given value (categorical split).
        """
        matching_idxs = np.argwhere(X_column == value).flatten()
        return matching_idxs

    def entropy(self, y):
        """
        Computes the entropy of the target labels.
        """
        counts = np.bincount(y)
        ps = counts / len(y)
        return -np.sum([p * np.log2(p) for p in ps if p > 0])

    def predict(self, X):
        """
        Predicts the class labels for the input data X.
        """
        return list(self.traverse_tree(x, self.root) for x in X.values)

    def traverse_tree(self, x, node):
        """
        Recursively traverses the tree to make a prediction for a single sample.
        """
        if node.is_leaf():
            return node.value

        if x[node.feature] in node.children:
            return self.traverse_tree(x, node.children[x[node.feature]])

        return node.value  # fallback if branch is missing


def accuracy(y_test, y_pred):
    """
    Calculates the accuracy of predictions.
    """
    return np.mean(y_test == y_pred) * 100


### Treinamento da árvore de decisão

Para separar os dados em conjuntos de treino e teste, utilizamos uma função auxiliar personalizada chamada custom_train_test_split, que simula o comportamento da função train_test_split da biblioteca scikit-learn.

In [8]:
def custom_train_test_split(df, test_size=0.3, shuffle=True, random_state=None):
    """
    Splits a DataFrame into training and testing sets using the last column as the label.

    Parameters:
    - df: pandas DataFrame with features and label (label must be the last column).
    - test_size: proportion of the dataset to include in the test split.
    - shuffle: whether to shuffle before splitting.
    - random_state: seed for reproducibility.

    Returns:
    - X_train, X_test, y_train, y_test
    """
    if shuffle:
        df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)

    # Identify the label column (last column)
    label_column = df.columns[-1]

    cutoff = int((1 - test_size) * len(df))
    train_df = df.iloc[:cutoff, :]
    test_df = df.iloc[cutoff:, :]

    X_train = train_df.drop(columns=label_column)
    y_train = train_df[label_column]

    X_test = test_df.drop(columns=label_column)
    y_test = test_df[label_column]

    return X_train, X_test, y_train, y_test


#### Dataset Iris

Antes de aplicarmos nossa árvore de decisão ao jogo Connect Four, realizamos um teste com o famoso dataset Iris. Esse conjunto de dados contém amostras de três tipos de flores (Setosa, Virginica e Versicolor), descritas por quatro atributos numéricos: comprimento e largura das pétalas e sépalas.

Como a implementação da árvore de decisão trabalha com valores discretos, realizamos uma discretização por quartis nos atributos contínuos, dividindo cada coluna em 3 categorias (0, 1 e 2).

Em seguida, dividimos o conjunto em 70% para treino e 30% para teste, treinamos a árvore com profundidade máxima 5 e avaliamos sua acurácia.

In [9]:
import pandas as pd
import numpy as np

# Discretize features using quantiles
def discretize_by_quantiles(df, n_bins=3):
    df_discrete = pd.DataFrame()
    for column in df.columns:
        df_discrete[column] = pd.qcut(df[column], q=n_bins, labels=False, duplicates='drop')
    return df_discrete

# Load the Iris dataset
iris = pd.read_csv("datasets/iris.csv")

# Separate features and class
X = iris.drop("class", axis=1)
y = iris["class"]

# Manually encode class labels as integers
class_mapping = {label: idx for idx, label in enumerate(sorted(y.unique()))}
y_encoded = y.map(class_mapping)


X_discrete = discretize_by_quantiles(X, n_bins=3)

# Rebuild the full DataFrame with encoded class as the last column
iris_discrete = pd.concat([X_discrete, pd.Series(y_encoded, name="class")], axis=1)

# Use custom train/test split
X_train, X_test, y_train, y_test = custom_train_test_split(
    iris_discrete, test_size=0.3, shuffle=True, random_state=42
)

# Train the decision tree
tree = DecisionTree(max_depth=5)
tree.fit(X_train, y_train)

# Predict
y_pred = tree.predict(X_test)

# Evaluate
acc = accuracy(y_test.values, y_pred)
print(f"Accuracy on Iris test set: {acc:.2f}%")


Accuracy on Iris test set: 100.00%


A árvore de decisão implementada obteve uma acurácia de 100% no conjunto de teste do dataset Iris. Todos os exemplos foram classificados corretamente, o que indica que a lógica da construção da árvore, os critérios de divisão e a discretização dos dados funcionaram conforme o esperado.

No entanto, é importante destacar que o Iris é um dataset simples, com classes bem separadas e número reduzido de atributos. Por isso, embora o resultado seja positivo, ele não garante que o modelo terá o mesmo desempenho em cenários mais complexos, como no Connect Four, onde os estados e decisões são muito mais variados e estratégicos.

#### Dataset Connect4

##### Preprocessamento

In [17]:
import pandas as pd

# Load the Connect Four dataset
df = pd.read_csv("datasets\connect4_dataset.csv")

# Remove duplicate rows (if any)
df = df.drop_duplicates()

In [20]:
import pandas as pd
import numpy as np

# Separate features and label
x = df.drop("chosen_move", axis=1)
y = df["chosen_move"]

# Define K-Fold parameters
k = 5
n = len(df)
fold_size = n // k

accuracies = []

print("Evaluating model using 5-Fold Cross-Validation:\n")

for fold_idx in range(k):
    start = fold_idx * fold_size
    end = (fold_idx + 1) * fold_size if fold_idx < k - 1 else n

    # Split data into training and testing sets
    X_train = pd.concat([x.iloc[:start], x.iloc[end:]])
    Y_train = pd.concat([y.iloc[:start], y.iloc[end:]])

    X_test = x.iloc[start:end]
    Y_test = y.iloc[start:end]

    # Train the model
    tree = DecisionTree(max_depth=5)
    tree.fit(X_train, Y_train)

    # Make predictions
    y_pred = tree.predict(X_test)

    # Evaluate accuracy
    acc = accuracy(Y_test.values, y_pred)
    accuracies.append(acc)

    print(f"Fold {fold_idx + 1}/{k} - Accuracy: {acc:.2f}%")

# Final results
mean_acc = np.mean(accuracies)
std_acc = np.std(accuracies)

print("\nFinal Evaluation:")
print(f"Average accuracy over {k} folds: {mean_acc:.2f}%")
print(f"Standard deviation: ±{std_acc:.2f}%")

Evaluating model using 5-Fold Cross-Validation:

Fold 1/5 - Accuracy: 25.45%
Fold 2/5 - Accuracy: 24.36%
Fold 3/5 - Accuracy: 24.87%
Fold 4/5 - Accuracy: 24.68%
Fold 5/5 - Accuracy: 26.29%

Final Evaluation:
Average accuracy over 5 folds: 25.13%
Standard deviation: ±0.68%


In [24]:
# Apply custom hold-out split (80% train, 20% test)
X_train, X_test, y_train, y_test = custom_train_test_split(df, test_size=0.3, shuffle=True, random_state=42)

# Train the decision tree
tree = DecisionTree(max_depth=5)
tree.fit(X_train, y_train)

# Evaluate on test set
y_pred = tree.predict(X_test)
acc = accuracy(y_test.values, y_pred)

print(f"Hold-out accuracy using custom split: {acc:.2f}%")


Hold-out accuracy using custom split: 29.87%


#### More information:

The best split is determined by iterating through all the possible columns and the unique values of each column. For each feature have at most 3 different values, the data is split based on whether the feature's value matches the given value. Information gain is then calculated by computing the entropy of the resulting splits, those that match the value and those that don’t. The best split is the feature that gives the highest information gain.

# **Conclusão**

In this project, we built a Connect 4 game from scratch in Python. Throughout the process, we explored ways to optimize game logic and performance, implemented the Monte Carlo Tree Search (MCTS) algorithm, and developed an intuitive algorithm interface. Additionally, we also learnt how to creat a dataset and built a decision tree model that is easy to interpret.

We are particularly proud of our MCTS implementation, which is capable of running up to 20000 simulations in just a few seconds. The game supports three game modes: Human vs Human, Human vs AI, and AI vs AI, with adjustable AI difficulty by tuning the number of MCTS iterations.

The decision tree model performed better than initially expected. Although we started with an accuracy of 47%, after experimented with various split criteria and configurations, the model achieved an improved accuracy of 50.97%.

To further validate the model, we used 5-fold cross-validation, which revealed variations in accuracy performance across different folds:
- Fold 1: 52.27%
- Fold 2: 54.11%
- Fold 3: 47.02%
- Fold 4: 49.68%
- Fold 5: 50.97%

These results demonstrate that cross-validation helped us uncover the potential of our decision tree beyond the initial test set.
